In [ ]:
#building a snake game using pygame
#importing required libraries
import pygame
import random
import csv
import os
import string

#initializing pygame
pygame.init()

#Constants for the game
WIDTH = 800
HEIGHT = 600
BLOCK_SIZE = 20
BLACK = (0, 0, 0)
GREEN = (0, 255, 0)
RED = (255, 0, 0)
WHITE = (255, 255, 255)
BLUE = (0, 100, 255)
GOLD = (255, 215, 0)

#display
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption('Snake Game :)')
clock = pygame.time.Clock()
font = pygame.font.SysFont(None, 48)
small_font = pygame.font.SysFont(None, 32)
tiny_font = pygame.font.SysFont(None, 24)

HIGHSCORES_PLAYER_FILE = 'highscores.csv'
PLAYER_DATA_FILE = 'player_data.csv'

#different snake types
snakes = {
    0: {'name': 'Classic Green', 'color': GREEN, 'start_len': 3, 'speed_bonus': 0, 'doubloons_mult': 1, 'price': 0},
    1: {'name': 'Speedy Red', 'color': RED, 'start_len': 3, 'speed_bonus': 3, 'doubloons_mult': 1, 'price': 100},
    2: {'name': 'Bulky Blue', 'color': BLUE, 'start_len': 5, 'speed_bonus': -1, 'doubloons_mult': 1, 'price': 200},
    3: {'name': 'Golden', 'color': GOLD, 'start_len': 4, 'speed_bonus': 1, 'doubloons_mult': 2, 'price': 500}
}

#creating the highscores file
def create_highscores_file():
    if not os.path.exists(HIGHSCORES_PLAYER_FILE):
        with open(HIGHSCORES_PLAYER_FILE, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(['Name', 'Score'])

#loading highscores
def load_highscores():
    scores = []
    try:
        with open(HIGHSCORES_PLAYER_FILE, 'r') as f:
            reader = csv.reader(f)
            next(reader)  # Skip header
            for row in reader:
                if len(row) >= 2:
                    scores.append((row[0], int(row[1])))
    except:
        pass
    return sorted(scores, key=lambda x: x[1], reverse=True)[:10]

#saving highscores
def save_highscore(name, score):
    scores = load_highscores()
    scores.append((name, score))
    scores = sorted(scores, key=lambda x: x[1], reverse=True)[:10]
    with open(HIGHSCORES_PLAYER_FILE, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['Name', 'Score'])
        writer.writerows(scores)

#loading saved player data
def load_player_data():
    if not os.path.exists(PLAYER_DATA_FILE):
        return {'doubloons': 50, 'selected': 0, 'unlocked': {0}}
    try:
        with open(PLAYER_DATA_FILE, 'r') as f:
            reader = csv.reader(f)
            row = next(reader)
            doubloons = int(row[0])
            selected = int(row[1])
            unlocked_str = row[2] if len(row) > 2 else ''
            unlocked = set(map(int, unlocked_str.split(','))) if unlocked_str else {0}
            return {'doubloons': doubloons, 'selected': selected, 'unlocked': unlocked}
    except:
        return {'doubloons': 50, 'selected': 0, 'unlocked': {0}}

#saving player data    
def save_player_data(data):
    unlocked_str = ','.join(map(str, sorted(data['unlocked'])))
    with open(PLAYER_DATA_FILE, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow([data['doubloons'], data['selected'], unlocked_str])

#text on screen
def draw_text(text, font_object, color, x, y):
    surf = font_object.render(text, True, color)
    rect = surf.get_rect(center=(x, y))
    screen.blit(surf, rect)

#main menu for the snake game
def main_menu(data):
    while True:
        screen.fill(BLACK)
        draw_text('Snake Game', font, WHITE, WIDTH // 2, 100)
        draw_text(f'doubloons: {data["doubloons"]}', small_font, GREEN, WIDTH // 2, 160)
        draw_text(f"Current Snake: {snakes[data['selected']]['name']}", small_font, BLUE, WIDTH // 2, 200)
        draw_text('SPACE to Play', small_font, BLUE, WIDTH // 2, 260)
        draw_text('S to access Shop', small_font, WHITE, WIDTH // 2, 300)
        draw_text('H for High Scores', small_font, WHITE, WIDTH // 2, 340)
        draw_text('Q to Quit', small_font, WHITE, WIDTH // 2, 380)
        pygame.display.flip()
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                return 'quit'
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_SPACE:
                    return 'play'
                if event.key == pygame.K_s:
                    return 'shop'
                if event.key == pygame.K_h:
                    show_highscores()
                if event.key == pygame.K_q:
                    return 'quit'
        clock.tick(60)

#Display of highscores
def show_highscores():
    highscores = load_highscores()
    while True:
        screen.fill(BLACK)
        draw_text('HIGH SCORES ;)', font, WHITE, WIDTH // 2, 100)
        draw_text('SPACE To Go Back', small_font, BLUE, WIDTH // 2, HEIGHT - 100)
        for i, (n, s) in enumerate(highscores):
            color = GREEN if i < 3 else WHITE
            draw_text(f' {i+1}. {n}: {s}', small_font, color, WIDTH // 2, 200 + i * 40)
        pygame.display.flip()
        for event in pygame.event.get():
            if event.type == pygame.QUIT or (event.type == pygame.KEYDOWN and event.key == pygame.K_SPACE):
                return
        clock.tick(60)

#shop for buying and selecting snakes
def shop_menu(data):
    snakes_list = list(snakes.keys())
    shop_sel = 0
    while True:
        screen.fill(BLACK)
        draw_text('SS (SNAKE SHOP)', font, WHITE, WIDTH // 2, 80)
        draw_text(f'doubloons: {data["doubloons"]}', small_font, GREEN, WIDTH // 2, 130)
        for j, i in enumerate(snakes_list):
            st = snakes[i]
            owned = i in data['unlocked']
            text = st['name']
            if j == shop_sel:
                text = '> ' + text
            if owned:
                text += ' (Owned)'
            else:
                text += f' - {st["price"]} coins'
            color = BLUE if j == shop_sel else GREEN if owned else WHITE
            draw_text(text, small_font, color, WIDTH // 2, 200 + j * 50)
        draw_text('UP/DOWN: Navigate  ENTER: Buy/Select  SPACE: Back', tiny_font, WHITE, WIDTH // 2, HEIGHT - 80)
        pygame.display.flip()
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                return data
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_UP:
                    shop_sel = (shop_sel - 1) % len(snakes_list)
                elif event.key == pygame.K_DOWN:
                    shop_sel = (shop_sel + 1) % len(snakes_list)
                elif event.key == pygame.K_RETURN:
                    i = snakes_list[shop_sel]
                    if i in data['unlocked']:
                        data['selected'] = i
                        save_player_data(data)
                    else:
                        price = snakes[i]['price']
                        if data['doubloons'] >= price:
                            data['doubloons'] -= price
                            data['unlocked'].add(i)
                            data['selected'] = i
                            save_player_data(data)
                elif event.key == pygame.K_SPACE:
                    return data
        clock.tick(60)

#Player name for highscore
def get_name(score):
    name = ''
    blink = 0
    while True:
        blink += 1
        screen.fill(BLACK)
        draw_text(f'New High Score: {score}!', font, GREEN, WIDTH // 2, HEIGHT // 2 - 100)
        draw_text('Enter Name (max 10 characters):', small_font, WHITE, WIDTH // 2, HEIGHT // 2 - 30)
        display_name = name + '|' if blink % 60 < 30 else name
        draw_text(display_name, font, WHITE, WIDTH // 2, HEIGHT // 2 + 30)
        draw_text('ENTER to Save / BACKSPACE to delete', tiny_font, WHITE, WIDTH // 2, HEIGHT // 2 + 80)
        pygame.display.flip()
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                return 'PLAYER'
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_RETURN:
                    return (name or 'PLAYER').upper()
                if event.key == pygame.K_BACKSPACE:
                    name = name[:-1]
                elif len(name) < 10 and event.unicode.isalpha():
                    name += event.unicode.upper()
        clock.tick(60)

#game over screen
def game_over(score):
    highscores = load_highscores()
    is_new_high = len(highscores) < 10 or score > highscores[-1][1]
    name = 'PLAYER'
    if is_new_high:
        name = get_name(score)
        save_highscore(name, score)
        highscores = load_highscores()  

    while True:
        screen.fill(BLACK)
        draw_text('GAME OVER', font, RED, WIDTH // 2, HEIGHT // 2 - 120)
        draw_text(f'Score: {score}', small_font, WHITE, WIDTH // 2, HEIGHT // 2 - 70)
        draw_text('Top Scores:', small_font, WHITE, WIDTH // 2, HEIGHT // 2 - 20)
        for i, (n,s) in enumerate(highscores):
            color = GREEN if n == name and s == score else WHITE
            draw_text(f' {i+1}. {n}: {s}', tiny_font, color, WIDTH // 2, HEIGHT // 2 + 20 + i * 25)
        draw_text('SPACE TO MENU', small_font, BLUE, WIDTH // 2, HEIGHT - 100)
        pygame.display.flip()
        for event in pygame.event.get():
            if event.type == pygame.QUIT or (event.type == pygame.KEYDOWN and event.key == pygame.K_SPACE):
                return
        clock.tick(60)
                
#Playing the game
def play_game(snake_type):
    color = snake_type['color']
    start_len = snake_type['start_len']
    speed_bonus = snake_type['speed_bonus']

    #snake body and initial settings
    head_x = (WIDTH // 2 // BLOCK_SIZE) * BLOCK_SIZE
    head_y = (HEIGHT // 2 // BLOCK_SIZE) * BLOCK_SIZE
    snake = [(head_x - i * BLOCK_SIZE, head_y) for i in range(start_len)]
    dx, dy = BLOCK_SIZE, 0
    score = 0
    level = 1
    base_speed = 8 + speed_bonus

    #food spawning

    def spawn_food():
        while True:
            food_x = (random.randrange(0, WIDTH - BLOCK_SIZE) // BLOCK_SIZE) * BLOCK_SIZE
            food_y = (random.randrange(0, HEIGHT - BLOCK_SIZE) // BLOCK_SIZE) * BLOCK_SIZE
            if (food_x, food_y) not in snake:
                return food_x, food_y

    food_x, food_y = spawn_food()

    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                return 0
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_LEFT and dx != BLOCK_SIZE:
                    dx, dy = -BLOCK_SIZE, 0
                elif event.key == pygame.K_RIGHT and dx != -BLOCK_SIZE:
                    dx, dy = BLOCK_SIZE, 0
                elif event.key == pygame.K_UP and dy != BLOCK_SIZE:
                    dx, dy = 0, -BLOCK_SIZE
                elif event.key == pygame.K_DOWN and dy != -BLOCK_SIZE:
                    dx, dy = 0, BLOCK_SIZE

        # movement and growth of snake
        head_x = snake[-1][0] + dx
        head_y = snake[-1][1] + dy
        new_head = (head_x, head_y)
        snake.append(new_head)

        # Food spawning
        if head_x == food_x and head_y == food_y:
            score += 10
            food_x, food_y = spawn_food()
        else:
            snake.pop(0)

        # Level and speed increasing
        new_level = (score // 50) + 1
        if new_level > level:
            level = new_level
        speed = base_speed + (level - 1) * 2

        # Collisions detection
        if head_x < 0 or head_x >= WIDTH or head_y < 0 or head_y >= HEIGHT or new_head in snake[:-1]:
            return score

        # Draw everything
        screen.fill(BLACK)
        pygame.draw.rect(screen, RED, [food_x, food_y, BLOCK_SIZE, BLOCK_SIZE])
        for segment in snake:
            pygame.draw.rect(screen, color, [segment[0], segment[1], BLOCK_SIZE, BLOCK_SIZE])
        draw_text(f'Score: {score}  Level: {level}', small_font, WHITE, 60, 30)
        pygame.display.flip()
        clock.tick(speed)

def main():
    create_highscores_file()
    data = load_player_data()
    while True:
        choice = main_menu(data)
        if choice == 'quit':
            break
        elif choice == 'play':
            snake_type = snakes[data['selected']]
            score = play_game(snake_type)
            doubloons_earned = (score // 10) * snake_type['doubloons_mult']
            data['doubloons'] += doubloons_earned
            save_player_data(data)
            game_over(score)
        elif choice == 'shop':
            shop_menu(data)
    pygame.quit()

if __name__ == '__main__':
    main()
